In [2]:
#pip install keras_tuner

In [1]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import keras_tuner as kt
import numpy as np
import argparse
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from pandas import read_csv
import math

In [3]:
OUTPUT_PATH = "output"
INPUT_SHAPE = (13, 21, 1)
NUM_CLASSES = 2
EPOCHS = 100
BS = 64
EARLY_STOPPING_PATIENCE = 5

In [4]:
df1 = pd.read_csv('recon650k.csv')
df2 = pd.read_csv('labels650k.csv')

X = df1.values
y = df2.values

n = 650000

X = np.reshape(X, (n,13,21,1))

df2.shape()

In [5]:
df1.shape

(650000, 273)

In [6]:
df2.drop('x-entry', axis=1, inplace=True)
df2.drop('y-entry', axis=1, inplace=True)
df2.drop('z-entry', axis=1, inplace=True)
df2.drop('n_x', axis=1, inplace=True)
df2.drop('n_y', axis=1, inplace=True)
df2.drop('n_z', axis=1, inplace=True)
df2.drop('number_eh_pairs', axis=1, inplace=True)

print(df2.head())
y = df2.values

   cotAlpha   cotBeta
0  3.696619  0.098554
1  2.341443 -0.206051
2  7.428552 -0.017690
3  7.807505  0.044182
4  3.911512 -0.191527


In [7]:
def build_model(hp):
	model = Sequential()
    
	inputShape = INPUT_SHAPE
	chanDim = -1

	model.add(Conv2D(hp.Int("conv_1", min_value=32, max_value=96, step=32), (3, 3), padding="same", input_shape=inputShape))
	model.add(Activation("relu"))
	model.add(MaxPooling2D(pool_size=(2, 2)))
	model.add(Conv2D(
	hp.Int("conv_2", min_value=64, max_value=128, step=32),(3, 3), padding="same"))
	model.add(Activation("relu"))
	model.add(Flatten())
	model.add(Dense(hp.Int("dense_units", min_value=64,
		max_value=512, step=256)))
	model.add(Activation("relu"))
	model.add(Dropout(0.1))
	model.add(Dense(NUM_CLASSES))
	model.add(Activation("linear"))
	lr = hp.Choice("learning_rate",
		values=[1e-1, 1e-2, 1e-3])
	opt = Adam(learning_rate=lr)
	model.compile(optimizer=opt, loss="mean_squared_error",
		metrics=["mean_squared_error"])
	return model

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.20, random_state = 0)
print(X.shape, y.shape)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

#scale input data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
X_test = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)

es = EarlyStopping(
	monitor="mean_squared_error",
	patience=EARLY_STOPPING_PATIENCE,
	restore_best_weights=True)


#tuner = kt.RandomSearch( build_model, objective="mean_squared_error", max_trials=10, seed=32, directory=OUTPUT_PATH,

tuner = kt.BayesianOptimization(build_model, objective="mean_squared_error", max_trials=30, seed=32, directory=OUTPUT_PATH, project_name='bayes')

print("[INFO] performing hyperparameter search...")
tuner.search(
  x=X_train, y=y_train, validation_data=(X_test, y_test), batch_size=BS, callbacks=[es], epochs=EPOCHS
	)

bestHP = tuner.get_best_hyperparameters(num_trials=1)[0]
print("[INFO] optimal number of filters in conv_1 layer: {}".format(bestHP.get("conv_1")))
print("[INFO] optimal number of filters in conv_2 layer: {}".format(bestHP.get("conv_2")))
print("[INFO] optimal number of units in dense layer: {}".format(bestHP.get("dense_units")))
print("[INFO] optimal learning rate: {:.4f}".format(bestHP.get("learning_rate")))

Trial 12 Complete [01h 03m 59s]
mean_squared_error: 0.01579948514699936

Best mean_squared_error So Far: 0.01561440248042345
Total elapsed time: 07h 39m 06s

Search: Running Trial #13

Hyperparameter    |Value             |Best Value So Far 
conv_1            |64                |96                
conv_2            |96                |64                
dense_units       |320               |320               
learning_rate     |0.001             |0.001             

Epoch 1/100
8125/8125 [==============================] - 58s 7ms/step - loss: 0.0571 - mean_squared_error: 0.0571 - val_loss: 0.0200 - val_mean_squared_error: 0.0200
Epoch 2/100
8125/8125 [==============================] - 56s 7ms/step - loss: 0.0249 - mean_squared_error: 0.0249 - val_loss: 0.0116 - val_mean_squared_error: 0.0116
Epoch 3/100
3028/8125 [==========>...................] - ETA: 31s - loss: 0.0229 - mean_squared_error: 0.0229